In [2]:
import networkx as nx
import pandas as pd
import numpy as np

In [3]:
import os
username = os.environ['JUPYTERHUB_USER']

In [4]:
from wybe import *
%load_ext autoreload
%autoreload 2

# Assumptions

We chose to make the following additional assumption to perform the routing :

* Maximum waiting time of 45 minutes at a stop
* Only connections between 6 AM and 10 PM are considered
* add additional delay to make connection. Ex 1 min 30 are needed to exit train and station (or change of track)


# Data loading and graph creation

Using the set of connections (called edges here) computed in the `Spark` notebooks, we will build the transport network using stops as nodes. Since there are multiple schedules between two stops, we shall use a directed multi-edge graph.

Each node maps to a stop in the 20km Zurich area. They are identified by their `stop id` (keys in the graph) and also contain the following properties :

* `name`: Name of the stop 
* `lat` and `lon`  :  the GPS coordinates of the stop.

Regarding the edges (connections between two stops : `A -> B`), we considererd two types :

* Standard connections (refered as `edges` in the code below)       : connections that are definied in the timetable data.
* Foot     connections (refered as `edges_foot` in the code below)  : walking transfer between any two stops that are at a distance of at most 500 m from each other.

Both types will include tthe following additional properties in the graph :

* `dep_time` : departure time  at stop `A` (timestamp)
* `arr_time` : arrival   time  at stop `B` (timestamp) 
* `gamma`    : array of parameters of the gamma distribution modeling the delay of the connection 
* `ttype`    : transport type used  (e.g. Bus, S-Bahn, etc.) 
* `travel_time`: the travel time of the connection 
* `trip_id`  : identifier of the trip 


In the following cells, we will load the data stored in the cluster and create the graph

> Note that since we want to be able to query for a fixed arrival time, the edges stored in the graph are reversed compared to the `real` connection.

> Furthermore,  for `Foot connections`, the fields `gamma` is irrelevant (set to `None`) and that we cannot specify `dep_time` and `arr_time` at creations time (set to 0 initially) but they will be dynamically field in the traversal of the graph according to the current time. 

In [33]:
def build_graph():
    """
    Read data from cluster and create transport graph
    /!\ Do no have to run this function since the graph is already stored in the cluster
    """
    edges = Utils.load_hdfs_to_pandas("edges_with_gammas_global_2.parquet").query("dep_time <= arr_time")
    edges = edges.rename(columns={
            'gammas':'gamma',
        'route_desc':'ttype',
        'start_lat':'dep_lat',
        'start_lon':'dep_lon',
    })
    edges             = Utils.format_edges(edges)
    edges_foot        = Utils.create_edges_foot(edges)
    G,stop_name_to_id = Utils.create_graph(edges,edges_foot)
    Utils.save_graph_to_hdfs(G, stop_name_to_id,suffix='2')

In [5]:
# read previously computed graph from the cluster
G,stop_name_to_id = Utils.read_graph_from_hdfs(suffix="2")

# Graph validation
We will first make sure that all stops in the 15km radius are connected as requested

In [6]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 2164
Number of edges: 999381
Average in degree: 461.8212
Average out degree: 461.8212


Since the graph is directed, we can use the strongly connected components algorithm to check for connectivity.

In [40]:
nx.is_strongly_connected(G)

False

However, we find that this is not the case at the global level. Note that it is not a problem since we are are only asked to find routes between two stops that fall within the 15 km radius. 
To perform this validation step, we will have to actually compute the strongly connected components and check that no stops ouside the max-size component are in the considered radius. 

In [41]:
components        = nx.strongly_connected_components(G)       # compute the connected components
sorted_components = sorted(components, key=len, reverse=True) # sort them by size
[len(c) for c in sorted_components]                           # display the sizes to get a better idea of the connectivity 

[2132, 9, 6, 4, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]

In [42]:
# construct the considered set of stops to perform efficient 'contains' queries
stops_data = Utils.load_hdfs_to_pandas("stops_15km.parquet")
stops_15km = set(stops_data.stop_id.values)

In [43]:
connected = True                       # intially we set the flag to True
for small_c in sorted_components[1:]:  # iterate over all components except the largest one
    for stop in small_c:               # iterate over all stops in these components
        if stop in stops_15km:         # if one of them is the radius we have a connectivity issue
            connected = False          # record this issue
print(f"All the stops within the 15 km radius are{'' if connected else ' not'} connected")

All the stops within the 15 km radius are connected


In [44]:
print(f" The claim that all stop names are unique is : {len(stops_data.stop_name.drop_duplicates()) == len(stops_data.stop_name)}")

 The claim that all stop names are unique is : True


# Testing

In [24]:
testRoutes = {
    'simple':            ('Zürich, ETH/Universitätsspital','Zürich Enge', '18:00:00',         0.9),
    'medium':            ('Opfikon','Stettbach', '12:00:00',                                  0.95),
    'medium-2':          ("Fällanden, Schützenhaus", "Wallisellen", '12:00:00',               0.9),
    'medium-high-proba': ("Zürich Wiedikon, Bahnhof","Zürich Flughafen, Bahnhof", '10:40:00', 0.22),
    'long'  :            ('Pfäffikon ZH, Wallikon','Bremgarten', '11:10:00',                  0.90)
}   

# Robust Routing validation

In [13]:
# Method to squeeze the dataframes to have the edges of the same trip represented in a single entry
def squeeze_df(df):
        # Get Intermediate stop for each trip in route dataframe
        intermediate_stops = df[['trip_id','dep_stop_name']].groupby('trip_id')\
                .agg({'dep_stop_name':lambda x : list(x)[1:]})\
                .reset_index().rename(columns={'dep_stop_name':'intermediate_stops'})
        
        # Select the values to be kept for each trip
        df = df.groupby('trip_id')\
                .agg({'ttype':'first','dep_stop_id':'first','arr_stop_id':'last','dep_time':'first','arr_time':'last',
                     'dep_stop_name':'first','arr_stop_name':'last','dep_lat':'first','dep_lon':'first','arr_lat':'last',
                      'arr_lon':'last','travel_time':'sum'})\
                .reset_index()
        # Merge stops with information about trip and return resulting dataframe
        return df.merge(intermediate_stops,on='trip_id').sort_values(by='dep_time')

In [32]:
routing = Routing(G, stop_name_to_id)
route_list = routing.robust('Zürich Flughafen','Zürich HB', '12:00:00',threshold = 0.9, verbose=True, number_of_routes=1)

Iteration : 1 || Proba : 0.9041965303961121


In [33]:
for r in route_list:
    display(squeeze_df(route_list[0].to_Pandas()))
    print(r)

,trip_id,ttype,dep_stop_id,arr_stop_id,dep_time,arr_time,dep_stop_name,arr_stop_name,dep_lat,dep_lon,arr_lat,arr_lon,travel_time,intermediate_stops
0,270.TA.6-8-j19-1.132.R,Intercity,8503016,8503000,42360.0,42900.0,Zürich Flughafen,Zürich HB,47.450381,8.562382,47.379271,8.540194,540.0,[]



        =========== Route info  =========
        Departure time      : 11:46
        Arrival   time      : 12:00
        ---------------------------------
        Success probability : 0.904
        Travel    time      : 00:14
        =========== Connections =========
        
            At stop : Zürich Flughafen and at 11:46
            take the Intercity 270.TA.6-8-j19-1.132.R to Zürich HB which arrives at 11:55
            Wait 5.0 min
            


<img src="https://i.imgur.com/Rzqfpnx.png">

Fortunately, for such a simple route, our algorithm finds the same trip as the SBB website. Note that in our output, the algorithm tells us to wait for 5 min at the end since we told him we wanted to arrive at 12:00 and it arrived a bit early. This is not an issue.

In [35]:
route_list = routing.robust('Opfikon','Stettbach', '12:00:00',threshold = 0.95, verbose=True, number_of_routes=1)
for r in route_list:
    display(squeeze_df(route_list[0].to_Pandas()))
    print(r)

Iteration : 1 || Proba : 0.9584134305240971


,trip_id,ttype,dep_stop_id,arr_stop_id,dep_time,arr_time,dep_stop_name,arr_stop_name,dep_lat,dep_lon,arr_lat,arr_lon,travel_time,intermediate_stops
0,0,Foot,8503340,8590629,40542.600191,40680.0,Opfikon,"Glattbrugg, Post",47.430014,8.561771,47.430537,8.561115,137.399809,[]
3,491.TA.26-768-j19-1.1.H,Bus,8590629,8591063,40680.000000,41040.0,"Glattbrugg, Post","Zürich Oerlikon, Bahnhof Ost",47.430537,8.561115,47.413336,8.545844,360.000000,"[Glattbrugg, Frohbühlstrasse, Zürich, Ettenfel..."
1,7,Foot,8591063,8503006,41268.280662,41640.0,"Zürich Oerlikon, Bahnhof Ost",Zürich Oerlikon,47.413336,8.545844,47.411835,8.544110,371.719338,[]
2,102.TA.26-19-j19-1.15.R,S-Bahn,8503006,8503000,41640.000000,41940.0,Zürich Oerlikon,Zürich HB,47.411835,8.544110,47.379271,8.540194,300.000000,[]
4,97.TA.26-12-j19-1.17.H,S-Bahn,8503000,8503147,42360.000000,42900.0,Zürich HB,Stettbach,47.379271,8.540194,47.397334,8.596132,480.000000,[Zürich Stadelhofen]



        =========== Route info  =========
        Departure time      : 11:15
        Arrival   time      : 12:00
        ---------------------------------
        Success probability : 0.958
        Travel    time      : 00:44
        =========== Connections =========
        
            At stop : Opfikon and at 11:15
            walk 114 m to Glattbrugg, Post which arrives at 11:18
            Wait 0.0 min
            
            At stop : Glattbrugg, Post and at 11:18
            take the Bus 491.TA.26-768-j19-1.1.H to Glattbrugg, Frohbühlstrasse which arrives at 11:19
            Wait 0.0 min
            
            At stop : Glattbrugg, Frohbühlstrasse and at 11:19
            take the Bus 491.TA.26-768-j19-1.1.H to Zürich, Ettenfeld which arrives at 11:20
            Wait 0.0 min
            
            At stop : Zürich, Ettenfeld and at 11:20
            take the Bus 491.TA.26-768-j19-1.1.H to Zürich, Seebach which arrives at 11:22
            Wait 0.0 min
            
    

<img src="https://i.imgur.com/DXnzA3t.png">

This route is a bit longer. In this case, the output is quite different : the SBB website tells us to leave 14 min later than our algorithm. Maybe the connection in Zürich Stadelhofen had a too high probability of not being feasible due to delays, so our algorithm chose a different route to satisfy the probability constraints.

In [42]:
route_list = routing.robust('Zürich Wiedikon, Bahnhof','Zürich Flughafen, Bahnhof', '12:00:00',threshold = 0.8, verbose=True, number_of_routes=1)
for r in route_list:
    display(squeeze_df(route_list[0].to_Pandas()))
    print(r)

Iteration : 1 || Proba : 0.8916902335646066


,trip_id,ttype,dep_stop_id,arr_stop_id,dep_time,arr_time,dep_stop_name,arr_stop_name,dep_lat,dep_lon,arr_lat,arr_lon,travel_time,intermediate_stops
3,116.TA.26-350-j19-1.4.R,Bus,8573710,8591341,40680.000000,40740.0,"Zürich Wiedikon, Bahnhof","Zürich, Schmiede Wiedikon",47.371430,8.524186,47.370152,8.519263,60.000000,[]
6,346.TA.26-72-j19-1.1.R,Bus,8591341,8591145,40980.000000,41160.0,"Zürich, Schmiede Wiedikon","Zürich, Friedhof Sihlfeld",47.370152,8.519263,47.375932,8.510891,180.000000,"[Zürich, Zwinglihaus, Zürich, Bertastrasse]"
0,4,Foot,8591145,8591038,41202.789109,41520.0,"Zürich, Friedhof Sihlfeld","Zürich, Albisriederplatz",47.375932,8.510891,47.378213,8.510396,317.210891,[]
7,370.TA.26-33-B-j19-1.1.R,Bus,8591038,8591060,41520.000000,41700.0,"Zürich, Albisriederplatz","Zürich Hardbrücke, Bahnhof",47.378213,8.510396,47.384934,8.517035,180.000000,"[Zürich, Hardplatz]"
1,7,Foot,8591060,8503020,41895.069405,41940.0,"Zürich Hardbrücke, Bahnhof",Zürich Hardbrücke,47.384934,8.517035,47.385256,8.517107,44.930595,[]
4,21.TA.26-9-A-j19-1.13.H,S-Bahn,8503020,8503006,41940.000000,42240.0,Zürich Hardbrücke,Zürich Oerlikon,47.385256,8.517107,47.411835,8.544110,300.000000,[]
5,327.TA.26-2-j19-1.141.R,S-Bahn,8503006,8503016,42540.000000,42840.0,Zürich Oerlikon,Zürich Flughafen,47.411835,8.544110,47.450381,8.562382,300.000000,[]
2,10,Foot,8503016,8573205,42948.187375,43200.0,Zürich Flughafen,"Zürich Flughafen, Bahnhof",47.450381,8.562382,47.450769,8.563747,251.812625,[]



        =========== Route info  =========
        Departure time      : 11:18
        Arrival   time      : 12:00
        ---------------------------------
        Success probability : 0.892
        Travel    time      : 00:42
        =========== Connections =========
        
            At stop : Zürich Wiedikon, Bahnhof and at 11:18
            take the Bus 116.TA.26-350-j19-1.4.R to Zürich, Schmiede Wiedikon which arrives at 11:19
            Wait 4.0 min
            
            At stop : Zürich, Schmiede Wiedikon and at 11:23
            take the Bus 346.TA.26-72-j19-1.1.R to Zürich, Zwinglihaus which arrives at 11:24
            Wait 0.0 min
            
            At stop : Zürich, Zwinglihaus and at 11:24
            take the Bus 346.TA.26-72-j19-1.1.R to Zürich, Bertastrasse which arrives at 11:25
            Wait 0.0 min
            
            At stop : Zürich, Bertastrasse and at 11:25
            take the Bus 346.TA.26-72-j19-1.1.R to Zürich, Friedhof Sihlfeld which a

<img src="https://i.imgur.com/oONXUdZ.png">

In this case, our route seems to be way too complicated compared to the results of the SBB website. It may be the case that the edge used by SBB is not present in our dataset (we checked and it happened for some edges before). In any case, our algorithm still finds a reliable way to get to its destination in a reasonable time.